In [1]:
%config Completer.use_jedi = False
import numpy as np
import pandas as pd
import os
import multiprocessing

cores_available = multiprocessing.cpu_count()

In [2]:
import pyproj
import lightgbm as lgb

from scipy import spatial

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from fuzzywuzzy import fuzz

In [3]:
def name_fuzz_ratio_comp (df_obj, name_col, indices_col):
    name_array = df_obj[name_col].to_numpy()
    return df_obj[['name','indices']].apply(
        lambda row: [fuzz.ratio(row['name'], index_name) 
                     for index_name 
                     in name_array[row['indices']]],
        axis=1)

In [4]:
def dataset_conversion(df: pd.DataFrame, transformer: pyproj.transformer.Transformer):
    df['name'].fillna(value='',inplace=True)
    
    geocentr_cartesian = df[[ 'latitude', 'longitude']].apply(lambda x: transformer.transform( x.longitude, x.latitude, 0, radians=False), axis=1).to_numpy()
    tree = spatial.KDTree(geocentr_cartesian.tolist())
    loc_data = tree.query(geocentr_cartesian.tolist(),min(len(df),10), workers=max(cores_available-1, 1))    
    distances, indices = loc_data
    
    df [["distances", "indices"]]=pd.DataFrame(
        data = {
         "distances": distances.tolist(),
         "indices": indices.tolist()
        }, 
        columns = ["distances", "indices"]
    )
    
    df['name_comp_fuzz_score'] = name_fuzz_ratio_comp(df, 'name', 'indices')
    
    x_train_set = np.apply_along_axis(
        lambda x: list(
            zip(
                np.repeat(
                    np.arange(x[0], x[0] + 1 ), len(x[1])
                ),
                x[1],
                x[2],
                x[3]
            )
        ),
        1, 
        df[['indices', 'distances', 'name_comp_fuzz_score']].reset_index().to_numpy()
    ).reshape((-1,4)).astype(int)
    
    try:
        y_train_set = (df['point_of_interest'].to_numpy()[x_train_set[:,0]]==df['point_of_interest'].to_numpy()[x_train_set[:,1]]).astype(int)
    except:
        y_train_set = None
    
    return x_train_set, y_train_set

In [5]:
transformer = pyproj.Transformer.from_crs(
        {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},
        {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
        )

In [6]:
dataframe_train_set = pd.read_csv('/kaggle/input/foursquare-location-matching/train.csv')

In [7]:
dataframe_test_set = pd.read_csv('/kaggle/input/foursquare-location-matching/test.csv')

In [8]:
x_train_set, y_train = dataset_conversion(dataframe_train_set, transformer)

In [9]:
x_test_set, _ = dataset_conversion(dataframe_test_set, transformer)

In [10]:
X_train, X_test = x_train_set[:,2:], x_test_set[:,2:]

In [11]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [12]:
# pd.read_csv('kaggle/input/foursquare-location-matching/pairs.csv')
# pd.read_csv('kaggle/input/foursquare-location-matching/test.csv')
# pd.read_csv('/kaggle/input/foursquare-location-matching/sample_submission.csv')
# pd.read_csv('/kaggle/input/foursquare-location-matching/train.csv')


In [13]:
d_train = lgb.Dataset(X_train, label=y_train)

In [14]:
params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 40
params['min_data'] = 50
params['max_depth'] = 10

In [15]:
clf = lgb.train(params, d_train, 1000)
y_pred=clf.predict(X_test)
y_pred[y_pred >= 0.5] = 1
y_pred[y_pred < 0.5] = 0

[LightGBM] [Info] Number of positive: 1802314, number of negative: 9585806
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.043505 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 355
[LightGBM] [Info] Number of data points in the train set: 11388120, number of used features: 2
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.158263 -> initscore=-1.671212
[LightGBM] [Info] Start training from score -1.671212


In [16]:
y_pred

array([1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 1., 0., 0., 0., 0.])

In [17]:
x_test_set

array([[       0,        0,        0,      100],
       [       0,        3,  1004977,       35],
       [       0,        4,  1005364,       38],
       [       0,        2,  7013604,        0],
       [       0,        1, 11766837,       34],
       [       1,        1,        0,      100],
       [       1,        2,  8001658,        0],
       [       1,        0, 11766837,       34],
       [       1,        3, 12068478,       23],
       [       1,        4, 12068751,       28],
       [       2,        2,        0,      100],
       [       2,        0,  7013604,        0],
       [       2,        3,  7813276,        0],
       [       2,        4,  7813427,        0],
       [       2,        1,  8001658,        0],
       [       3,        3,        0,      100],
       [       3,        4,     1008,       50],
       [       3,        0,  1004977,       35],
       [       3,        2,  7813276,        0],
       [       3,        1, 12068478,       23],
       [       4,   

In [32]:
y_pred.reshape((1,-1)).T

(25, 1)

In [31]:
x_test_set.shape

(25, 4)

In [34]:
np.concatenate((y_pred.reshape((1,-1)).T,x_test_set), axis=1).astype(int)

array([[       1,        0,        0,        0,      100],
       [       0,        0,        3,  1004977,       35],
       [       0,        0,        4,  1005364,       38],
       [       0,        0,        2,  7013604,        0],
       [       0,        0,        1, 11766837,       34],
       [       1,        1,        1,        0,      100],
       [       0,        1,        2,  8001658,        0],
       [       0,        1,        0, 11766837,       34],
       [       0,        1,        3, 12068478,       23],
       [       0,        1,        4, 12068751,       28],
       [       1,        2,        2,        0,      100],
       [       0,        2,        0,  7013604,        0],
       [       0,        2,        3,  7813276,        0],
       [       0,        2,        4,  7813427,        0],
       [       0,        2,        1,  8001658,        0],
       [       1,        3,        3,        0,      100],
       [       0,        3,        4,     1008,       50